In [ ]:
from correlations import *
from utils.various_tools import *
from utils.msa_manipulation import *
from utils.sample_MSA_from_EVE_model import *
from utils.compare_marginals import *
from higher_order_covariation import *

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.precision", 3)

MSA_data_folder='./data/MSA'
MSA_weights_location='./data/weights'
VAE_checkpoint_location='./results/VAE_parameters/'
correlations_location='./results/correlations'
mutations_location='./data/mutations'
evol_indices_location = "./results/evol_indices"
comparisons_location = "./results/correlations/_comparisons"

In this notebook we sample large $N$ = 100K MSAs from different models and compare their low- and high-order marginals.

## Mi3: Converting the generated MSA and computing its 1,2-point marginals

For 2-site models, we compute the 1- and 2-site marginals of an MSA generated by Mi3 after training for 60 runs on `PABP_38k.a2m`.

#### Run_60:

In [ ]:
input_MSA_filename = MSA_data_folder + os.sep + "Mi3_generated" + os.sep + "PABP_38k_run60_100k"
output_MSA_filename = MSA_data_folder + os.sep + "PABP_38k_run60_100k.a2m"
prefix = ">PABP_38k_run60_100k/Seq_"
focus_line = ">PABP_YEAST/115-210\nKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGQEIYVAPHLSR"

convert_MSA_Mi3_generated_to_EVE(input_MSA_filename = input_MSA_filename, 
                                 output_MSA_filename = output_MSA_filename, 
                                 N_out = 100*1000
                                 focus_line = focus_line, prefix = prefix)

In [ ]:
model_name = "PABP_38k_run60_100k"
MSA_location = MSA_data_folder + os.sep + model_name + ".a2m"
correlations_subfolder = correlations_location + os.sep + model_name + os.sep
shell_run('mkdir -p ' + correlations_subfolder)
file_name_label_to_seq_out = correlations_subfolder + model_name + "_label_to_seq.npy"
file_name_weights_out = MSA_weights_location + os.sep + model_name + ".npy"

Mi3_60 = Correlations(MSA_location = MSA_location, MFA = False,
                        file_name_label_to_seq_out = file_name_label_to_seq_out,
                        file_name_weights_out = file_name_weights_out,
                        advanced_preprocess_MSA=False
                       )

In [ ]:
Mi3_60.compute_all()
save_instance_to_file( correlations_subfolder + model_name + ".Correlations", Mi3_60)

## Sampling large MSAs from EVE and computing 1,2-point marginals:

Now we sample 100K sequences from EVE trained on `PABP_38k.a2m` after 400K steps and 1400K steps, and after 200K steps in non-Bayesian mode. We then compute their 1- and 2-site marginals and correlations.

In [ ]:
N_samples = 100*1000

#### EVE_400:

In [ ]:
n_steps = 400*1000
in_VAE_checkpoint_location = VAE_checkpoint_location + "PABP_38k" + os.sep + "PABP_38k_step_" + str(n_steps)

generate_MSA_and_Correlations_from_EVE_model(ref_weights_file_name = "PABP_38k",
                            ref_Correlations_name = "PABP_38k",
                            ref_MSA_name = "PABP_38k.a2m",
                            in_VAE_checkpoint_location = in_VAE_checkpoint_location,
                            out_Correlations_name = 'PABP_38k_VAE_' + str(n_steps//1000) + 'k_100k_NoThresholds',
                            N_samples = N_samples,
                            threshold = 0.3,
                            model_parameters_location='./EVE/default_model_params.json',
                            debug = False
                           );

#### EVE_1400:

In [ ]:
n_steps = 1400*1000
in_VAE_checkpoint_location = VAE_checkpoint_location + "PABP_38k" + os.sep + "PABP_38k_step_" + str(n_steps)

generate_MSA_and_Correlations_from_EVE_model(ref_weights_file_name = "PABP_38k",
                            ref_Correlations_name = "PABP_38k",
                            ref_MSA_name = "PABP_38k.a2m",
                            in_VAE_checkpoint_location = in_VAE_checkpoint_location,
                            out_Correlations_name = 'PABP_38k_VAE_' + str(n_steps//1000) + 'k_100k_NoThresholds',
                            N_samples = N_samples,
                            threshold = 0.3,
                            model_parameters_location='./EVE/default_model_params.json',
                            debug = False
                           );

#### VAE_NoBayes_200:

In [ ]:
n_steps = 200*1000
in_VAE_checkpoint_location = VAE_checkpoint_location + "PABP_38k" + os.sep + "PABP_38k_NoBayes_step_" + str(n_steps)

generate_MSA_and_Correlations_from_EVE_model(ref_weights_file_name = "PABP_38k",
                            ref_Correlations_name = "PABP_38k",
                            ref_MSA_name = "PABP_38k.a2m",
                            in_VAE_checkpoint_location = in_VAE_checkpoint_location,
                            out_Correlations_name = 'PABP_38k_VAE_NoBayes_' + str(n_steps//1000) + 'k_100k_NoThresholds',
                            N_samples = N_samples,
                            threshold = 0.3,
                            model_parameters_location='./EVE/default_model_params_NoBayes.json', 
                            debug = False
                           );
    

## 1,2-pt results:

We are now able to compute the Pearson r's of 1- and 2-site marginals across these models against those of the training set (`rs_df_training`) and of the test set (`rs_df_test`).

In [ ]:
list_models = [["test", "PABP_38k_test_100k"],
               ["training", "PABP_38k"],
               ["Mi3_60", "PABP_38k_run60_100k"],
               ["EVE_400" , "PABP_38k_VAE_400k_100k_NoThresholds"],
               ["EVE_1400",  "PABP_38k_VAE_1400k_100k_NoThresholds" ],
               ["EVE_NoBayes_200", "PABP_38k_VAE_NoBayes_200k_100k_NoThresholds" ],
              ]

observables = ["f1", "f2", "CM2", "MI"]

In [ ]:
rs_df_training = rs_observables_to_df(list_models, observables, ref_label = "training", 
    output_filename = comparisons_location + os.sep + "1n2pt_statistics_training_allmodels_38k_100k.csv")
rs_df_training

In [ ]:
rs_df_test = rs_observables_to_df(list_models, observables, ref_label = "test", 
    output_filename = comparisons_location + os.sep + "1n2pt_statistics_test_allmodels_38k_100k.csv")
rs_df_test

## Comparing 3,4,5-order covariation:

We compute higher-order marginals for these models, and compute the $r_{20}$'s against training and test sets. We initialise the Correlations objects from scratch using the MSA sampled for that model in label_to_seq dictionary format, and its weights. 1,2-point marginals and the other metrics previously computed for these sampled MSAs are not required to compute higher-order marginals, and not loading them saves memory.

In [ ]:
test_l2s = np.load("./results/correlations/PABP_38k_test_100k/PABP_38k_test_100k_label_to_seq.npy",
                        allow_pickle = True).item()
training_l2s = np.load("./results/correlations/PABP_38k/PABP_38k_label_to_seq.npy",
                        allow_pickle = True).item()
Mi3_60_l2s = np.load("./results/correlations/PABP_38k_run60_100k/PABP_38k_run60_100k_label_to_seq.npy",
                        allow_pickle = True).item()
EVE_400_l2s = np.load("./results/correlations/PABP_38k_VAE_400k_100k_NoThresholds/PABP_38k_VAE_400k_100k_NoThresholds_label_to_seq.npy",
                        allow_pickle = True).item()
EVE_1400_l2s = np.load("./results/correlations/PABP_38k_VAE_1400k_100k_NoThresholds/PABP_38k_VAE_1400k_100k_NoThresholds_label_to_seq.npy",
                        allow_pickle = True).item()
EVE_200NB_l2s = np.load("./results/correlations/PABP_38k_VAE_NoBayes_200k_100k_NoThresholds/PABP_38k_VAE_NoBayes_200k_100k_NoThresholds_label_to_seq.npy",
                        allow_pickle = True).item()

test_w = np.load("./data/weights/PABP_38k_test_100k.npy")
training_w = np.load("./data/weights/PABP_38k.npy")
Mi3_60_w = np.load("./data/weights/PABP_38k_run60_100k.npy")
EVE_400_w = np.load("./data/weights/PABP_38k_VAE_400k_100k_NoThresholds.npy")
EVE_1400_w = np.load("./data/weights/PABP_38k_VAE_1400k_100k_NoThresholds.npy")
EVE_200NB_w = np.load("./data/weights/PABP_38k_VAE_NoBayes_200k_100k_NoThresholds.npy")


In [ ]:
test = Correlations(label_to_seq = test_l2s, weights = test_w)
training = Correlations(label_to_seq = training_l2s, weights = training_w)
Mi3_60 = Correlations(label_to_seq = Mi3_60_l2s, weights = Mi3_60_w)

EVE_400 = Correlations(label_to_seq = EVE_400_l2s, weights = EVE_400_w)
EVE_1400 = Correlations(label_to_seq = EVE_1400_l2s, weights = EVE_1400_w)
EVE_NoBayes_200 = Correlations(label_to_seq = EVE_200NB_l2s, weights = EVE_200NB_w)

We assess higher-order marginals based on 3K sets of site positions at each order.

In [ ]:
list_models = [training,
               test,
               Mi3_60,     
               EVE_400,
               EVE_1400,
               EVE_NoBayes_200
              ]
N_samples = 3000

#### Order 3:

In [ ]:
compute_r20s(list_models, 3, "rs_training_order3_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 0)

In [ ]:
compute_r20s(list_models, 3, "rs_test_order3_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 1)

#### Order 4:

In [ ]:
compute_r20s(list_models, 4, "rs_training_order4_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 0)

In [ ]:
compute_r20s(list_models, 4, "rs_test_order4_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 1)

#### Order 5:

In [ ]:
compute_r20s(list_models, 5, "rs_training_order5_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 0)

In [ ]:
compute_r20s(list_models, 5, "rs_test_order5_3ksamples_allmodels_38k_100k",
                        N_samples = N_samples, ref_model_index = 1)

### Adding the results to a df:

In [ ]:
r_f3_training = np.load(comparisons_location + os.sep + "rs_training_order3_3ksamples_allmodels_38k_100k.npy")
r_f4_training = np.load(comparisons_location + os.sep + "rs_training_order4_3ksamples_allmodels_38k_100k.npy")
r_f5_training = np.load(comparisons_location + os.sep + "rs_training_order5_3ksamples_allmodels_38k_100k.npy")

r_f3_test = np.load(comparisons_location + os.sep + "rs_test_order3_3ksamples_allmodels_38k_100k.npy")
r_f4_test = np.load(comparisons_location + os.sep + "rs_test_order4_3ksamples_allmodels_38k_100k.npy")
r_f5_test = np.load(comparisons_location + os.sep + "rs_test_order5_3ksamples_allmodels_38k_100k.npy")


In [ ]:
rs_df_training = pd.read_csv(comparisons_location + os.sep + "1n2pt_statistics_training_allmodels_38k_100k.csv") 
rs_df_test = pd.read_csv(comparisons_location + os.sep + "1n2pt_statistics_test_allmodels_38k_100k.csv") 


In [ ]:
rs_df_training.loc[len(rs_df_training)] = ["f3"] + r_f3_training[0].tolist()
rs_df_training.loc[len(rs_df_training)] = ["f4"] + r_f4_training[0].tolist()
rs_df_training.loc[len(rs_df_training)] = ["f5"] + r_f5_training[0].tolist()

rs_df_test.loc[len(rs_df_test)] = ["f3"] + r_f3_test[0].tolist()
rs_df_test.loc[len(rs_df_test)] = ["f4"] + r_f4_test[0].tolist()
rs_df_test.loc[len(rs_df_test)] = ["f5"] + r_f5_test[0].tolist()


In [ ]:
rs_df_training

In [ ]:
rs_df_test

In [ ]:
rs_df_training.to_csv(comparisons_location + os.sep + "allpt_statistics_training_allmodels_38k_100k.csv", index=False)
rs_df_test.to_csv(comparisons_location + os.sep + "allpt_statistics_test_allmodels_red4_38k.csv", index=False)
